In [70]:
#import necessary packages
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
import time
import matplotlib.pyplot as plt

In [71]:
#import data
PL20_21_fixtures = pd.read_csv('Data/2020-21/Fixtures/PL20_21_features.csv')
PL20_21_lineups = pd.read_csv('Data/2020-21/Players/PL2020_lineups.csv')
PL20_21_player_stats = pd.read_csv('Data/2020-21/Players/PL2020_total.csv')
PL20_21_lineups['player_id'] = PL20_21_lineups['player'] + PL20_21_lineups['team'] + PL20_21_lineups['date'].astype(str)
PL20_21_lineups['game_id'] = PL20_21_lineups['team'] + PL20_21_lineups['date'].astype(str)
PL20_21_lineups
PL20_21_player_stats = PL20_21_player_stats.rename(columns={'player_ID':'player_id','game_ID':'game_id'})

In [72]:
PL20_21_player_stats

,Unnamed: 0,player_fl,player,date,team,team_rating,category,position_fl,minutes_fl,minutes_rating,...,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,opponent_team,selected,player_id,game_id
0,0,Aaron Connolly,Aaron Connolly,2020-09-14,Brighton & Hove Albion,Brighton & Hove Albion,Forward,FWD,45,46.0,...,1,0,0,0,55,True,5,32205,Aaron ConnollyBrighton & Hove Albion2020-09-14,Brighton & Hove Albion2020-09-14
1,1,Aaron Cresswell,Aaron Cresswell,2020-09-12,West Ham United,West Ham United,Defender,DEF,90,90.0,...,1,0,0,0,50,True,14,70232,Aaron CresswellWest Ham United2020-09-12,West Ham United2020-09-12
2,2,Aaron Mooy,0,2020-09-14,Brighton & Hove Albion,NaN,NaN,MID,0,NaN,...,0,0,0,0,50,True,5,2010,0Brighton & Hove Albion2020-09-14,Brighton & Hove Albion2020-09-14
3,3,Aaron Ramsdale,Aaron Ramsdale,2020-09-14,Sheffield United,Sheffield United,Goalkeeper,GK,90,90.0,...,1,0,0,0,50,True,20,462939,Aaron RamsdaleSheffield United2020-09-14,Sheffield United2020-09-14
4,4,Abdoulaye Doucouré,Abdoulaye Doucouré,2020-09-13,Everton,Everton,Midfield,MID,90,90.0,...,3,0,0,0,55,False,17,143040,Abdoulaye DoucouréEverton2020-09-13,Everton2020-09-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24376,24376,Aaron Ramsdale,Aaron Ramsdale,2021-05-23,Sheffield United,Sheffield United,Goalkeeper,GK,90,90.0,...,9,13618,17381,3763,46,True,4,162985,Aaron RamsdaleSheffield United2021-05-23,Sheffield United2021-05-23
24377,24377,Adrien Silva,0,2021-05-23,Leicester City,NaN,NaN,MID,0,NaN,...,0,-16,0,16,44,True,17,16396,0Leicester City2021-05-23,Leicester City2021-05-23
24378,24378,Sidnei Tavares,Sidnei Tavares,2021-05-23,Leicester City,NaN,NaN,MID,0,NaN,...,0,32,60,28,44,True,17,337,Sidnei TavaresLeicester City2021-05-23,Leicester City2021-05-23
24379,24379,Kalvin Phillips,Kalvin Phillips,2021-05-23,Leeds United,Leeds United,Midfield,MID,90,90.0,...,6,210,789,579,49,True,18,74008,Kalvin PhillipsLeeds United2021-05-23,Leeds United2021-05-23


In [73]:
PL20_21_fixtures

,Unnamed: 0,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,...,Elo_h_after,Elo_a_after,Elo_h_before,Elo_a_before,goalsDifference,shotsDifference,cornersDifference,ewmDifference,elo_difference,form_difference
0,0,E0,2020-09-12,12:30,Fulham,Arsenal,0,3,A,0,...,1485.00,1515.00,1500.00,1500.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,1,E0,2020-09-12,15:00,Crystal Palace,Southampton,1,0,H,1,...,1515.00,1485.00,1500.00,1500.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,2,E0,2020-09-12,17:30,Liverpool,Leeds United,4,3,H,3,...,1515.00,1485.00,1500.00,1500.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,3,E0,2020-09-12,20:00,West Ham United,Newcastle United,0,2,A,0,...,1485.00,1515.00,1500.00,1500.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,4,E0,2020-09-13,14:00,West Bromwich Albion,Leicester City,0,3,A,0,...,1485.00,1515.00,1500.00,1500.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,375,E0,2021-05-23,16:00,Liverpool,Crystal Palace,2,0,H,1,...,1591.89,1439.73,1582.38,1449.24,2.500000,16.500000,12.833333,19.313782,133.14,5.500000
376,376,E0,2021-05-23,16:00,Manchester City,Everton,5,0,H,2,...,1659.36,1508.70,1649.79,1518.27,2.166667,1.666667,6.000000,24.719180,131.52,1.666667
377,377,E0,2021-05-23,16:00,Sheffield United,Burnley,1,0,H,1,...,1358.16,1420.86,1338.90,1440.12,-4.833333,-8.333333,-12.500000,-19.400129,-101.22,-0.500000
378,378,E0,2021-05-23,16:00,West Ham United,Southampton,3,0,H,2,...,1560.00,1420.08,1549.98,1430.10,2.833333,-3.500000,-6.666667,18.155503,119.88,2.500000


In [74]:
def get_wk(fixtures):
    home = fixtures[['HomeTeam', 'Wk_h', 'game_home_ID']]
    away = fixtures[['AwayTeam', 'Wk_a', 'game_away_ID']]
    home = home.rename(columns={'HomeTeam':'Team','Wk_h':'wk', 'game_home_ID': 'game_id'})
    away = away.rename(columns={'AwayTeam':'Team','Wk_a':'wk', 'game_away_ID': 'game_id'})
    return home.append(away)
game_weeks = get_wk(PL20_21_fixtures)
game_weeks


C:\Users\giuli\AppData\Local\Temp\ipykernel_4408\392273660.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return home.append(away)


,Team,wk,game_id
0,Fulham,1,Fulham2020-09-12
1,Crystal Palace,1,Crystal Palace2020-09-12
2,Liverpool,1,Liverpool2020-09-12
3,West Ham United,1,West Ham United2020-09-12
4,West Bromwich Albion,1,West Bromwich Albion2020-09-13
...,...,...,...
375,Crystal Palace,38,Crystal Palace2021-05-23
376,Everton,38,Everton2021-05-23
377,Burnley,38,Burnley2021-05-23
378,Southampton,38,Southampton2021-05-23


In [75]:
PL20_21_lineups_tmp =  pd.merge(PL20_21_lineups, game_weeks[['game_id', 'wk']], on= 'game_id', how='left')
PL20_21_player_stats_tmp =  pd.merge(PL20_21_player_stats, game_weeks[['game_id', 'wk']], on= 'game_id', how='left')
PL20_21_player_stats_tmp

,Unnamed: 0,player_fl,player,date,team,team_rating,category,position_fl,minutes_fl,minutes_rating,...,transfers_balance,transfers_in,transfers_out,value,was_home,opponent_team,selected,player_id,game_id,wk
0,0,Aaron Connolly,Aaron Connolly,2020-09-14,Brighton & Hove Albion,Brighton & Hove Albion,Forward,FWD,45,46.0,...,0,0,0,55,True,5,32205,Aaron ConnollyBrighton & Hove Albion2020-09-14,Brighton & Hove Albion2020-09-14,1
1,1,Aaron Cresswell,Aaron Cresswell,2020-09-12,West Ham United,West Ham United,Defender,DEF,90,90.0,...,0,0,0,50,True,14,70232,Aaron CresswellWest Ham United2020-09-12,West Ham United2020-09-12,1
2,2,Aaron Mooy,0,2020-09-14,Brighton & Hove Albion,NaN,NaN,MID,0,NaN,...,0,0,0,50,True,5,2010,0Brighton & Hove Albion2020-09-14,Brighton & Hove Albion2020-09-14,1
3,3,Aaron Ramsdale,Aaron Ramsdale,2020-09-14,Sheffield United,Sheffield United,Goalkeeper,GK,90,90.0,...,0,0,0,50,True,20,462939,Aaron RamsdaleSheffield United2020-09-14,Sheffield United2020-09-14,1
4,4,Abdoulaye Doucouré,Abdoulaye Doucouré,2020-09-13,Everton,Everton,Midfield,MID,90,90.0,...,0,0,0,55,False,17,143040,Abdoulaye DoucouréEverton2020-09-13,Everton2020-09-13,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24376,24376,Aaron Ramsdale,Aaron Ramsdale,2021-05-23,Sheffield United,Sheffield United,Goalkeeper,GK,90,90.0,...,13618,17381,3763,46,True,4,162985,Aaron RamsdaleSheffield United2021-05-23,Sheffield United2021-05-23,38
24377,24377,Adrien Silva,0,2021-05-23,Leicester City,NaN,NaN,MID,0,NaN,...,-16,0,16,44,True,17,16396,0Leicester City2021-05-23,Leicester City2021-05-23,38
24378,24378,Sidnei Tavares,Sidnei Tavares,2021-05-23,Leicester City,NaN,NaN,MID,0,NaN,...,32,60,28,44,True,17,337,Sidnei TavaresLeicester City2021-05-23,Leicester City2021-05-23,38
24379,24379,Kalvin Phillips,Kalvin Phillips,2021-05-23,Leeds United,Leeds United,Midfield,MID,90,90.0,...,210,789,579,49,True,18,74008,Kalvin PhillipsLeeds United2021-05-23,Leeds United2021-05-23,38


In [76]:
def get_lineup(lineups_df, game_id):
    rslt_df = lineups_df[lineups_df['game_id'] == game_id]
    return rslt_df

get_lineup( PL20_21_lineups_tmp, 'Arsenal2020-09-12',)

,player,team,matchweek,date,start11,player_id,game_id,wk
18,Bernd Leno,Arsenal,1.0,2020-09-12,True,Bernd LenoArsenal2020-09-12,Arsenal2020-09-12,1
19,Héctor Bellerín,Arsenal,1.0,2020-09-12,True,Héctor BellerínArsenal2020-09-12,Arsenal2020-09-12,1
20,Kieran Tierney,Arsenal,1.0,2020-09-12,True,Kieran TierneyArsenal2020-09-12,Arsenal2020-09-12,1
21,Gabriel Dos Santos,Arsenal,1.0,2020-09-12,True,Gabriel Dos SantosArsenal2020-09-12,Arsenal2020-09-12,1
22,Alexandre Lacazette,Arsenal,1.0,2020-09-12,True,Alexandre LacazetteArsenal2020-09-12,Arsenal2020-09-12,1
23,Willian,Arsenal,1.0,2020-09-12,True,WillianArsenal2020-09-12,Arsenal2020-09-12,1
24,Pierre-Emerick Aubameyang,Arsenal,1.0,2020-09-12,True,Pierre-Emerick AubameyangArsenal2020-09-12,Arsenal2020-09-12,1
25,Ainsley Maitland-Niles,Arsenal,1.0,2020-09-12,True,Ainsley Maitland-NilesArsenal2020-09-12,Arsenal2020-09-12,1
26,Rob Holding,Arsenal,1.0,2020-09-12,True,Rob HoldingArsenal2020-09-12,Arsenal2020-09-12,1
27,Mohamed Elneny,Arsenal,1.0,2020-09-12,True,Mohamed ElnenyArsenal2020-09-12,Arsenal2020-09-12,1


In [77]:
def get_player_stats(player_stats, lineup, wk):
    tmp = pd.merge(player_stats, lineup[['player', 'start11']], on= 'player')
    df = tmp.drop(tmp[tmp.wk > wk].index)
    return df



lineup = get_lineup(PL20_21_lineups_tmp, 'Arsenal2020-09-12')
#lineup
get_player_stats(PL20_21_player_stats_tmp, lineup,4)

,Unnamed: 0,player_fl,player,date,team,team_rating,category,position_fl,minutes_fl,minutes_rating,...,transfers_in,transfers_out,value,was_home,opponent_team,selected,player_id,game_id,wk,start11
0,13,Ainsley Maitland-Niles,Ainsley Maitland-Niles,2020-09-12,Arsenal,Arsenal,Defender,DEF,90,90.0,...,0,0,50,False,8,112927,Ainsley Maitland-NilesArsenal2020-09-12,Arsenal2020-09-12,1,True
1,441,Ainsley Maitland-Niles,Ainsley Maitland-Niles,2020-09-19,Arsenal,NaN,NaN,DEF,0,NaN,...,27762,14619,50,True,19,152759,Ainsley Maitland-NilesArsenal2020-09-19,Arsenal2020-09-19,2,True
2,980,Ainsley Maitland-Niles,Ainsley Maitland-Niles,2020-09-28,Arsenal,Arsenal,Defender,DEF,90,90.0,...,5689,36098,50,False,11,123510,Ainsley Maitland-NilesArsenal2020-09-28,Arsenal2020-09-28,3,True
3,1535,Ainsley Maitland-Niles,Ainsley Maitland-Niles,2020-10-04,Arsenal,Arsenal,Midfield,DEF,3,4.0,...,4867,14107,50,True,15,119317,Ainsley Maitland-NilesArsenal2020-10-04,Arsenal2020-10-04,4,True
38,16,Alexandre Lacazette,Alexandre Lacazette,2020-09-12,Arsenal,Arsenal,Forward,FWD,86,86.0,...,0,0,85,False,8,196064,Alexandre LacazetteArsenal2020-09-12,Arsenal2020-09-12,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,2010,Sead Kolasinac,Sead Kolašinac,2020-10-04,Arsenal,NaN,NaN,DEF,0,NaN,...,764,2139,49,True,15,16501,Sead KolašinacArsenal2020-10-04,Arsenal2020-10-04,4,False
646,414,Willian Borges Da Silva,Willian,2020-09-12,Arsenal,Arsenal,Forward,MID,74,75.0,...,0,0,80,False,8,473242,WillianArsenal2020-09-12,Arsenal2020-09-12,1,True
647,951,Willian Borges Da Silva,Willian,2020-09-19,Arsenal,Arsenal,Forward,MID,63,63.0,...,474433,67202,81,True,19,1119588,WillianArsenal2020-09-19,Arsenal2020-09-19,2,True
648,1505,Willian Borges Da Silva,Willian,2020-09-28,Arsenal,Arsenal,Forward,MID,67,67.0,...,81422,334961,81,False,11,858445,WillianArsenal2020-09-28,Arsenal2020-09-28,3,True


In [50]:
test =  pd.merge(PL20_21_lineups_tmp, PL20_21_player_stats_tmp[['player_id', 'category', 'position', 'minutes', 'rating', 'xP', 'bonus', 'bps', 'ict_index', 'influence', 'creativity', 'threat', 'total_points', 'transfers_in', 'transfers_out', 'transfers_balance', 'value', 'was_home']], on= 'player_id', how='left')
test

,player,team,matchweek,date,start11,player_id,game_id,wk,category,position,...,ict_index,influence,creativity,threat,total_points,transfers_in,transfers_out,transfers_balance,value,was_home
0,Marek Rodák,Fulham,1.0,2020-09-12,True,Marek RodákFulham2020-09-12,Fulham2020-09-12,1,NaN,GK,...,1.8,18.2,0.0,0.0,1.0,0.0,0.0,0.0,45.0,True
1,Michael Hector,Fulham,1.0,2020-09-12,True,Michael HectorFulham2020-09-12,Fulham2020-09-12,1,Defender,CB,...,1.7,16.8,0.6,0.0,0.0,0.0,0.0,0.0,45.0,True
2,Denis Odoi,Fulham,1.0,2020-09-12,True,Denis OdoiFulham2020-09-12,Fulham2020-09-12,1,Defender,RB,...,2.1,11.4,2.6,7.0,1.0,0.0,0.0,0.0,45.0,True
3,Neeskens Kebano,Fulham,1.0,2020-09-12,True,Neeskens KebanoFulham2020-09-12,Fulham2020-09-12,1,Forward,RW,...,2.2,1.0,2.7,18.0,2.0,0.0,0.0,0.0,50.0,True
4,Tom Cairney,Fulham,1.0,2020-09-12,True,Tom CairneyFulham2020-09-12,Fulham2020-09-12,1,Midfield,DM,...,1.7,7.0,10.2,0.0,1.0,0.0,0.0,0.0,55.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14641,Michael Obafemi,Southampton,38.0,2021-05-23,False,Michael ObafemiSouthampton2021-05-23,Southampton2021-05-23,38,Forward,FW,...,0.0,0.0,0.3,0.0,1.0,618.0,520.0,98.0,46.0,False
14642,Nathan Tella,Southampton,38.0,2021-05-23,False,Nathan TellaSouthampton2021-05-23,Southampton2021-05-23,38,Forward,FW,...,2.5,4.6,1.4,19.0,1.0,1899.0,1061.0,838.0,43.0,False
14643,Theo Walcott,Southampton,38.0,2021-05-23,False,Theo WalcottSouthampton2021-05-23,Southampton2021-05-23,38,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14644,Dan Nlundulu,Southampton,38.0,2021-05-23,False,Dan NlunduluSouthampton2021-05-23,Southampton2021-05-23,38,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
PL20_21_lineups_p = PL20_21_lineups[['player', 'team']]
PL20_21_lineups_p= PL20_21_lineups_p.drop_duplicates()
PL20_21_lineups_p

,player,team
0,Marek Rodák,Fulham
1,Michael Hector,Fulham
2,Denis Odoi,Fulham
3,Neeskens Kebano,Fulham
4,Tom Cairney,Fulham
...,...,...
14441,Hannibal Mejbri,Manchester United
14442,William Thomas Fish,Manchester United
14523,Tim Iroegbunam,West Bromwich Albion
14541,Tyrese Francois,Fulham
